# Data processer
## Proyecto final -- Optimización y Simulación
### Alejandro de Haro, Jose María Benítez, Daniel Escobosa, Pablo Berástegui

In [43]:
import pandas as pd

In [44]:
df_players = pd.read_csv('../data/male_players.csv')

/var/folders/pr/nst7vndn0l10vqp0s73wclhr0000gn/T/ipykernel_82078/910873479.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_players = pd.read_csv('../data/male_players.csv')


Sólo queremos el año 18

In [45]:
df_players = df_players[df_players['fifa_version'] == 18.0]

In [46]:
# tipos de cada columna
for col in df_players.columns:
    print(f"{col}: {df_players[col].dtype}")

player_id: int64
player_url: object
fifa_version: float64
fifa_update: float64
update_as_of: object
short_name: object
long_name: object
player_positions: object
overall: int64
potential: int64
value_eur: float64
wage_eur: float64
age: int64
dob: object
height_cm: int64
weight_kg: int64
club_team_id: float64
club_name: object
league_id: float64
league_name: object
league_level: float64
club_position: object
club_jersey_number: float64
club_loaned_from: object
club_joined_date: object
club_contract_valid_until_year: float64
nationality_id: int64
nationality_name: object
nation_team_id: float64
nation_position: object
nation_jersey_number: float64
preferred_foot: object
weak_foot: int64
skill_moves: int64
international_reputation: int64
work_rate: object
body_type: object
real_face: object
release_clause_eur: float64
player_tags: object
player_traits: object
pace: float64
shooting: float64
passing: float64
dribbling: float64
defending: float64
physic: float64
attacking_crossing: int64
at

In [47]:
df_players.shape

(17954, 109)

In [48]:
all_pos = df_players.player_positions.unique()
positions = set()
for pos in all_pos:
    for p in pos.split(','):
        positions.add(p.strip())

positions = list(positions)
print(positions)

['RM', 'LW', 'ST', 'CM', 'CF', 'LM', 'LWB', 'LB', 'GK', 'RB', 'CB', 'RWB', 'CDM', 'CAM', 'RW']


In [49]:
df_players['position'] = df_players['player_positions'].apply(lambda x: x.split(',')[0].strip())

In [50]:
df_clean = df_players.copy()

# quedarse solo con las columnas relevantes

relevant_cols = ['player_id', 'short_name', 'overall', 'value_eur', 'club_team_id','club_name', 'position', 'league_id', 'league_name', 'nationality_id','nationality_name', 'pace','shooting','passing','dribbling','defending','physic']
df_clean = df_clean[relevant_cols]
df_clean.head()
                 

,player_id,short_name,overall,value_eur,club_team_id,club_name,position,league_id,league_name,nationality_id,nationality_name,pace,shooting,passing,dribbling,defending,physic
111583,20801,Cristiano Ronaldo,94,95500000.0,243.0,Real Madrid,LW,53.0,La Liga,38,Portugal,90.0,93.0,82.0,90.0,33.0,80.0
111584,158023,L. Messi,93,105000000.0,241.0,FC Barcelona,RW,53.0,La Liga,52,Argentina,89.0,90.0,86.0,96.0,26.0,61.0
111585,190871,Neymar Jr,92,123000000.0,73.0,Paris Saint Germain,LW,16.0,Ligue 1,54,Brazil,92.0,84.0,79.0,95.0,30.0,60.0
111586,167495,M. Neuer,92,61000000.0,21.0,FC Bayern München,GK,19.0,Bundesliga,21,Germany,NaN,NaN,NaN,NaN,NaN,NaN
111587,176580,L. Suárez,92,97000000.0,241.0,FC Barcelona,ST,53.0,La Liga,60,Uruguay,82.0,90.0,79.0,87.0,42.0,81.0


In [51]:
print(len(df_clean))
len(df_clean[df_clean['position'] == 'GK'])

17954


2019

In [52]:
df_clean = df_clean[df_clean['position'].eq('GK') | df_clean.notna().all(axis=1)]

df_clean = df_clean.fillna(0)

In [53]:
len(df_clean)

17753

**Cogemos un sample de prueba**

In [54]:
targets = {
    "GK": 50,
    "CB": 100,
    "CM": 150,
    "RB": 50,
    "LB": 50,
    "RW": 50,
    "LW": 50,
    "ST": 50
}

In [55]:
sampled_dfs = []

for pos, n in targets.items():
    subset = df_clean[df_clean["position"] == pos]
    sample = subset.sample(n=n, random_state=42)
    sampled_dfs.append(sample)

df_sample = pd.concat(sampled_dfs, ignore_index=True)

df_sample.to_csv("../data/players_fifa18_sample.csv", index=False)

In [56]:
df_clean.to_csv('../data/players_fifa18_clean.csv', index=False)

In [57]:
def quimica(player_1, player_2):
    chemistry = 0
    
    if player_1['league_id'] == player_2['league_id']:
        chemistry += 1
        if player_1['club_team_id'] == player_2['club_team_id']:
            chemistry += 1
    if player_1['nationality_id'] == player_2['nationality_id']:
        chemistry += 1
    
    return chemistry

Posibles links en una 4-3-3

In [58]:
links = {
    'GK': ['CD', 'CI'],
    'RB': ['CD', 'MD'],
    'LB': ['CI', 'MI'],
    'CD': ['MD'],
    'CI': ['MI'],
    'CM': ['MD', 'MI', 'ST'],
    'MI': ['LW'],
    'MD': ['RW'],
    'RW': ['ST'],
    'LW': ['ST']
}

In [59]:
from random import randint

In [60]:
def reestructure_position(position):
    
    if position == 'CB':
        if randint(0,1) == 0:
            return 'CI'
        else:
            return 'CD'
    elif position == 'CM':
        r = randint(0,2)
        if r == 0:
            return 'MI'
        elif r == 1:
            return 'CM'
        else: 
            return 'MD'
    return position

In [61]:
df_sample['position'] = df_sample['position'].apply(lambda x: reestructure_position(x))

In [62]:
print(len(df_sample[df_sample['position'] == 'MI']))
print(len(df_sample[df_sample['position'] == 'CM']))
print(len(df_sample[df_sample['position'] == 'MD']))

52
41
57


In [67]:
df_sample.to_csv("../data/players_fifa18_sample.csv", index=False)

In [63]:
def pair_chemistry(df_i, df_j):
    # producto cartesiano
    df_i = df_i.rename(columns=lambda c: c + '_i')
    df_j = df_j.rename(columns=lambda c: c + '_j')

    df_i['key'] = 1
    df_j['key'] = 1
    pairs = df_i.merge(df_j, on='key').drop(columns='key')

    # reglas de química
    same_club   = pairs['club_team_id_i']   == pairs['club_team_id_j']
    same_league = pairs['league_id_i'] == pairs['league_id_j']
    same_nation = pairs['nationality_id_i'] == pairs['nationality_id_j']

    # sumar como enteros
    chem = same_league.astype(int) + same_club.astype(int) + same_nation.astype(int)
    pairs['chem'] = chem

    # eliminar parejas consigo mismo
    pairs = pairs[pairs['player_id_i'] != pairs['player_id_j']].copy()

    # forzar orden player_id_i < player_id_j
    p_min = pairs[['player_id_i', 'player_id_j']].min(axis=1)
    p_max = pairs[['player_id_i', 'player_id_j']].max(axis=1)
    pairs['player_id_i'] = p_min
    pairs['player_id_j'] = p_max

    # eliminar duplicados si (i,j) aparece varias veces
    pairs = pairs.drop_duplicates(subset=['player_id_i', 'player_id_j'])

    return pairs[['player_id_i', 'player_id_j', 'chem']]

**Químicas del sample**

In [64]:

results = []

for pos_i, neighs in links.items():
    df_i = df_sample[df_sample['position'] == pos_i]
    for pos_j in neighs:
        df_j = df_sample[df_sample['position'] == pos_j]
        if len(df_i) == 0 or len(df_j) == 0:
            continue
        res = pair_chemistry(df_i, df_j)
        results.append(res)

sample_chemistry_edges = pd.concat(results, ignore_index=True).drop_duplicates(subset=['player_id_i', 'player_id_j'])

In [65]:
sample_chemistry_edges.to_csv("../data/sample_chemistry_edges_fifa18.csv", index=False)

**Químicas reales; tarda mucho en ejecutar**

**SÓLO EJECUTAR SI NO ESTÁ DISPONIBLE CHEMISTRY_EDGES_FIFA18.CSV**

**LA EJECUCIÓN DURA 8MINS**

In [ ]:
results = []

for pos_i, neighs in links.items():
    df_i = df_players[df_players['position'] == pos_i]
    for pos_j in neighs:
        df_j = df_players[df_players['position'] == pos_j]
        if len(df_i) == 0 or len(df_j) == 0:
            continue
        res = pair_chemistry(df_i, df_j)
        results.append(res)

chemistry_edges = pd.concat(results, ignore_index=True).drop_duplicates(subset=['player_id_i', 'player_id_j'])

In [ ]:
chemistry_edges.to_csv('../data/chemistry_edges_fifa18.csv', index=False)

**SEGUIR EJECUTANDO AQUÍ**

In [26]:
chems = pd.read_csv('../data/chemistry_edges_fifa18.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/chemistry_edges_fifa18.csv'